***
# <font color=red>Building and Explaining a Text Classifier using AutoMLx</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle AutoMLx Team </font></p>

***

AutoMLx Text Classification Demo version 23.1.1.

Copyright (c) 2023 Oracle, Inc.  

Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl/

## Overview of this Notebook

In this notebook we will build a classifier using the Oracle AutoMLx tool for the public 20newsgroup dataset. The dataset is a binary classification dataset, and more details about the dataset can be found at http://qwone.com/~jason/20Newsgroups/.
We explore the various options provided by the Oracle AutoMLx tool, allowing the user to exercise control over the AutoML training process. We then evaluate the different models trained by AutoML. Finally we provide an overview of the possibilites that Oracle AutoMLx offers for explaining the predictions of the tuned model.

---
## Prerequisites

  - Experience level: Novice (Python and Machine Learning)
  - Professional experience: Some industry experience

Compatible conda pack: [Oracle AutoML and Model Explanation for Python 3.8](link TBD)

---

## Business Use

Data analytics and modeling problems using Machine Learning (ML) are becoming popular and often rely on data science expertise to build accurate ML models. Such modeling tasks primarily involve the following steps:
- Preprocess dataset (clean, impute, engineer features, normalize).
- Pick an appropriate model for the given dataset and prediction task at hand.
- Tune the chosen model’s hyperparameters for the given dataset.

All of these steps are significantly time consuming and heavily rely on data scientist expertise. Unfortunately, to make this problem harder, the best feature subset, model, and hyperparameter choice widely varies with the dataset and the prediction task. Hence, there is no one-size-fits-all solution to achieve reasonably good model performance. Using a simple Python API, AutoML can quickly (faster) jump-start the datascience process with an accurately-tuned model and appropriate features for a given prediction task.

## Table of Contents

- <a href='#setup'>0. Setup</a>
- <a href='#load-data'>1. Load the 20newsgroup Income dataset</a>
- <a href='#AutoML'>2. AutoML</a>
  - <a href='#Engine'>2.0. Set the engine</a>
  - <a href='#provider'>2.1. Create an Instance of Oracle AutoMLx</a>
  - <a href='#default'>2.2. Train a Model using AutoML</a>
  - <a href='#analyze'>2.3. Analyze the AutoML optimization process </a>
      - <a href='#algorithm-selection'>2.3.1. Algorithm Selection</a>
      - <a href='#adaptive-sampling'>2.3.2. Adaptive Sampling</a>
      - <a href='#feature-selection'>2.3.3. Feature Selection</a>
      - <a href='#hyperparameter-tuning'>2.3.4. Hyperparameter Tuning</a>
  - <a href='#timebudget'>2.4. Specify a time budget to AutoML</a>
  - <a href='#scoringfn'>2.5. Specify a different scoring metric to AutoML</a>
- <a href='#MLX'>3. Machine Learning Explainability (MLX)</a>
  - <a href='#MLX-initialization'> 3.1. Initialize an MLExplainer</a>
  - <a href='#MLX-global'> 3.2. Global Token Importance</a>
  - <a href='#MLX-local'> 3.3. Local Token Importance</a>
- <a href='#ref'>References</a>

<a id='setup'></a>
## Setup

Basic setup for the Notebook.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

Load the required modules.

In [ ]:
import gzip
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups

# Settings for plots
plt.rcParams['figure.figsize'] = [10, 7]
plt.rcParams['font.size'] = 15
sns.set(color_codes=True)
sns.set(font_scale=1.5)
sns.set_palette("bright")
sns.set_style("whitegrid")

import automl
from automl import init

<a id='load-data'></a>
## Load the 20 News Group dataset
We start by reading in the dataset from sklearn. The dataset has already been pre-split into training and test sets. The training set will be used to create a Machine Learning model using AutoML, and the test set will be used to evaluate the model's performance on unseen data.

In [ ]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

target_names = train.target_names

X_train, y_train = pd.DataFrame(train.data), pd.DataFrame(train.target)
X_test, y_test = pd.DataFrame(test.data), pd.DataFrame(test.target)

column_names = ["Message"]
X_train.columns = column_names
X_test.columns = column_names

Lets look at a few of the values in the data. 20 NewsGroup is a classification dataset made of text samples. Each sample has an associated class (also called topic), which can be one of the followings:

In [ ]:
train.target_names

We display some examples of data samples.

In [ ]:
X_train.head()

We restrict the data to the 4 topics that are related to science.

In [ ]:
science_labels = [i for i in range(len(target_names)) if 'sci' in target_names[i]]
train_science_indices = [i for i in range(len(y_train)) if y_train.iloc[i][0] in science_labels]
test_science_indices = [i for i in range(len(y_test)) if y_test.iloc[i][0] in science_labels]

X_train = X_train.iloc[train_science_indices]
y_train = y_train.iloc[train_science_indices]
X_test = X_test.iloc[test_science_indices]
y_test = y_test.iloc[test_science_indices]

In [ ]:
len(X_train)

We further downsample the train set to have a reasonable training time for this demonstration.

In [ ]:
X_train, _, y_train, _ = train_test_split(X_train, y_train,
                                          test_size=0.5,
                                          stratify=y_train,
                                          random_state=42)

Finally we generate a validation set and only use that for internal pipeline validation.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,
                                                      test_size=0.2,
                                                      stratify=y_train,
                                                      random_state=42)

<a id='AutoML'></a>
## AutoML

<a id='Engine'></a>
### Setting the engine
The AutoML pipeline offers the function `init`, which allows to initialize the parallel engine. By default, the AutoML pipeline uses the `dask` parallel engine. One can also set the engine to `local`, which uses python's multiprocessing library for parallelism instead.

In [ ]:
init(engine='local')

<a id='provider'></a>
### Create an instance of Oracle AutoMLx

The Oracle AutoMLx solution provides a pipeline that automatically finds a tuned model given a prediction task and a training dataset. In particular it allows to find a tuned model for any supervised prediction task, e.g. classification or regression where the target can be binary, categorical or real-valued.

AutoML consists of five main modules:
- **Preprocessing** (Feature extraction and selection) : The pipeline extracts tabular features using [TFIDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) that it then selects between.
- **Algorithm Selection** : Identify the right classification algorithm -in this notebook- for a given dataset, choosing from amongst:
    - AdaBoostClassifier
    - DecisionTreeClassifier
    - ExtraTreesClassifier
    - TorchMLPClassifier
    - KNeighborsClassifier
    - LGBMClassifier
    - LinearSVC
    - LogisticRegression
    - RandomForestClassifier
    - SVC
    - XGBClassifier
    - GaussianNB
- **Adaptive Sampling** : Select a subset of the data samples for the model to be trained on.
- **Feature Selection** : Select a subset of the features (extracted with TFIDF), based on the previously selected model. In the rest of this notebook, we will implicitly refer to the extracted features as data features.
- **Hyperparameter Tuning** : Find the right model parameters that maximize score for the given dataset.

All these pieces are readily combined into a simple AutoML pipeline which automates the entire Machine Learning process with minimal user input/interaction.

<a id='default'></a>
### Train a model using AutoML

The AutoML API is quite simple to work with. We create an instance of the pipeline. Next, the training data is passed to the `fit()` function which executes the three previously mentioned steps.

A model is then generated and can be used for prediction tasks. We use the roc_auc scoring metric to evaluate the performance of this model on unseen data (`X_test`).

In [ ]:
est1 = automl.Pipeline(task='classification')
est1.fit(X_train, y_train, X_valid, y_valid, cv=None, col_types=['text'])

y_predict = est1.predict(X_test)
score_default = f1_score(y_test, y_predict, average="micro")

print('F1 Micro Score on test data: {:3.3f}'.format(score_default))

<a id='analyze'></a>
### Analyze the AutoML optimization process

During the AutoML process, a summary of the optimization process is logged. It consists of:
- Information about the training data
- Information about the AutoML Pipeline, such as:
    - selected features that AutoML found to be most predictive in the training data;
    - selected algorithm that was the best choice for this data;
    - hyperparameters for the selected algorithm.

AutoML provides a print_summary API to output all the different trials performed.

In [ ]:
est1.print_summary()

We also provide the capability to visualize the results of each stage of the AutoML pipeline. 

<a id='algorithm-selection'></a>
#### Algorithm Selection

The plot below shows the scores predicted by Algorithm Selection for each algorithm. The horizontal line shows the average score across all algorithms. Algorithms below the line are colored turquoise, whereas those with a score higher than the mean are colored teal. Here we can see that the `TorchMLPClassifier` achieved the highest predicted score (orange bar), and is chosen for subsequent stages of the Pipeline.

In [ ]:
# Each trial is a tuple of
# (algorithm, no. samples, no. features, mean CV score, hyperparameters, 
# all CV scores, total CV time (s), memory usage (Gb))
trials = est1.model_selection_trials_
colors = []
scores = [x[3] for x in trials]
models = [x[0] for x in trials]
y_margin = 0.10 * (max(scores) - min(scores))
s = pd.Series(scores, index=models).sort_values(ascending=False)

for f in s.keys():
    if f == '{}_AS'.format(est1.selected_model_):
        colors.append('orange')
    elif s[f] >= s.mean():
        colors.append('teal')
    else:
        colors.append('turquoise')
        

fig, ax = plt.subplots(1)
ax.set_title("Algorithm Selection Trials")
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.set_ylabel(est1.inferred_score_metric[0])
s.plot.bar(ax=ax, color=colors, edgecolor='black')
ax.axhline(y=s.mean(), color='black', linewidth=0.5)
plt.show()

<a id='adaptive-sampling'></a>
#### Adaptive Sampling

Following Algorithm Selection, Adaptive Sampling aims to find the smallest dataset sample that can be created without compromising validation set score for the chosen model. Given the small size of the training data (948 samples), Adaptive Sampling is not relevant here.

In [ ]:
# Each trial is a tuple of
# (algorithm, no. samples, no. features, mean CV score, hyperparameters, 
# all CV scores, total CV time (s), memory usage (Gb))
trials = est1.adaptive_sampling_trials_
scores = [x[3] for x in trials]
n_samples = [x[1] for x in trials]
y_margin = 0.10 * (max(scores) - min(scores))

fig, ax = plt.subplots(1)
ax.set_title("Adaptive Sampling ({})".format(trials[0][0]))
ax.set_xlabel('Dataset sample size')
ax.set_ylabel(est1.inferred_score_metric[0])
ax.grid(color='g', linestyle='-', linewidth=0.1)
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.plot(n_samples, scores, 'k:', marker="s", color='teal', markersize=3)
plt.show()

<a id='feature-selection'></a>
#### Feature Selection
After finding a sample subset, the next step is to find a relevant feature subset to maximize score for the chosen algorithm. The Feature Selection step identifies the smallest feature subset that does not compromise on the score of the chosen algorithm. The orange line shows the optimal number of features chosen by Feature Selection (in this case, retaining only about 4 000 of the more than 11 000 available words).

In [ ]:
# Each trial is a tuple of
# (algorithm, no. samples, no. features, mean CV score, hyperparameters, 
# all CV scores, total CV time (s), memory usage (Gb))
trials = est1.feature_selection_trials_
scores = [x[3] for x in trials]
n_features = [x[2] for x in trials]
y_margin = 0.10 * (max(scores) - min(scores))


fig, ax = plt.subplots(1)
ax.set_title("Feature Selection Trials")
ax.set_xlabel("Number of Features")
ax.set_ylabel(est1.inferred_score_metric[0])
ax.grid(color='g', linestyle='-', linewidth=0.1)
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.plot(n_features, scores, 'k:', marker="s", color='teal', markersize=3)
ax.axvline(x=len(est1.selected_features_names_), color='orange', linewidth=2.0)
plt.show()

<a id='hyperparameter-tuning'></a>
#### Hyperparameter Tuning

Hyperparameter Tuning is the last stage of the AutoML pipeline, and focuses on improving the chosen algorithm's score on the reduced dataset (after Adaptive Sampling and Feature Selection). We use a novel algorithm to search across many hyperparameters dimensions, and converge automatically when optimal hyperparameters are identified. Each trial in the graph below represents a particular hyperparameters configuration for the selected model.

In [ ]:
# Each trial is a tuple of
# (algorithm, no. samples, no. features, mean CV score, hyperparameters, 
# all CV scores, total CV time (s), memory usage (Gb))
trials = est1.tuning_trials_
scores = [x[3] for x in reversed(trials)]
y_margin = 0.10 * (max(scores) - min(scores))


fig, ax = plt.subplots(1)
ax.set_title("Hyperparameter Tuning Trials")
ax.set_xlabel("Iteration $n$")
ax.set_ylabel(est1.inferred_score_metric[0])
ax.grid(color='g', linestyle='-', linewidth=0.1)
ax.set_ylim(min(scores) - y_margin, max(scores) + y_margin)
ax.plot(range(1, len(trials) + 1), scores, 'k:', marker="s", color='teal', markersize=3)
plt.show()

<a id='timebudget'></a>
### Specify a time budget to AutoML
The Oracle AutoMLx tool also supports a user given time budget in seconds. Given the small size of this dataset, we give a small `time budget` of 10 seconds using the time_budget argument.

In [ ]:
est2 = automl.Pipeline(task="classification")
est2.fit(X_train, y_train, X_valid, y_valid, cv=None, col_types=['text'], time_budget=10)

y_predict = est2.predict(X_test)
score_default = f1_score(y_test, y_predict, average="micro")

print('F1 micro Score on test data: {:3.3f}'.format(score_default))

<a id='scoringstr'></a>
### Specify a different scoring metric to AutoML
By default, the score metric is set to `neg_log_loss` for classifcation and `neg_mean_squared_error` for regression.

The user can also choose another scoring metric. The list of possible metrics is given by:
- For binary classification, one of 'roc_auc', 'accuracy', 'f1', 'precision', 'recall', 'f1_micro', 'f1_macro', 'f1_weighted', 'f1_samples', 'recall_micro', 'recall_macro', 'recall_weighted', 'recall_samples', 'precision_micro', 'precision_macro', 'precision_weighted', 'precision_samples'
- For multiclass classification , one of  'neg_log_loss', 'recall_macro', 'accuracy','f1_micro', 'f1_macro', 'f1_weighted', 'f1_samples', 'recall_micro', 'recall_weighted', 'recall_samples', 'precision_micro', 'precision_macro', 'precision_weighted', 'precision_samples'
- For regression, one of 'neg_mean_squared_error', 'r2', 'neg_mean_absolute_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error'

Here, we ask AutoML to optimize for the 'f1_micro' scoring metric.

In [ ]:
est3 = automl.Pipeline(task="classification", score_metric='f1_micro')
est3.fit(X_train, y_train, X_valid, y_valid, cv=None, col_types=['text'], time_budget=60)

y_predict = est3.predict(X_test)
score_default = f1_score(y_test, y_predict, average="micro")

print('F1 micro Score on test data: {:3.3f}'.format(score_default))

<a id='MLX'></a>
## Machine Learning Explainability

For a variety of decision-making tasks, getting only a prediction as model output is not sufficient. A user may wish to know why the model outputs that prediction, or which data features are relevant for that prediction. For that purpose the Oracle AutoMLx solution defines the MLExplainer object, which allows to compute a variety of model explanations

<a id='MLX-initializing'></a>
### Initializing an MLExplainer

The MLExplainer object takes as argument the trained model, the training data and labels, as well as the task.

In [ ]:
explainer = automl.MLExplainer(est1,
                               X_train,
                               y_train,
                               target_names=['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space'],
                               task="classification",
                               col_types=["text"])

<a id='MLX-global'></a>
### Model Explanations (Global Token Importance)

For text classification tasks, since we first extract tokens (features), the Oracle AutoMLx solution offers a single way to compute a notion of token importance: Global Token Importance. The notion of Global Token Importance intuitively measures how much a token impacts the model's predictions (relative to the provided train labels). This notion of token importance considers each token independently from all other tokens. Tokens are the most fine-grained building blocks of the NLP model, such as sentences, words, or characters.

#### Computing the importance

We use a permutation-based method to successively measure the importance of each token (feature). Such a method therefore runs in linear time with respect to the
number of features (tokens) in the dataset. 

The method `explain_model()` allows to compute such feature importances. It also provides 95% confidence intervals for each token importance attribution.

In [ ]:
result_explain_model_default = explainer.explain_model()

#### Visualization

There are two options to show the explanation's results:
- `to_dataframe()` will return a dataframe of the results.
- `show_in_notebook()` will show the results as a bar plot.

The features are returned in decreasing order of importance.

In [ ]:
result_explain_model_default.to_dataframe(n_tokens=20)

In [ ]:
result_explain_model_default.show_in_notebook(n_tokens=20)

<a id='MLX-local'></a>
## Local Token importance

For text classification tasks, since we first extract tokens (features), the Oracle AutoMLx solution offers a single way to compute a notion of token importance: Local Token Importance. The notion of Local Token Importance intuitively measures how much a token impacts an instance's predictions (relative to the provided train labels). This notion of token importance considers each token independently from all other tokens.

#### Compute the importance

By default we use a surrogate method to successively measure the importance of each token in a given instance. Such a method therefore runs in linear time with respect to the number of features in the dataset.

The method `explain_prediction()` allows to compute such feature importances. It also provides 95% confidence intervals for each feature importance attribution.

In [ ]:
index = 0
result_explain_prediction_default = explainer.explain_prediction(X_train.iloc[index:index + 1, :])

There are two options to show the explanation's results:
 - `to_dataframe()` will return a dataframe of the results.
 - `show_in_notebook()` will show the results as a bar plot.

The features are returned in decreasing order of importance.

#### Visualization

In [ ]:
result_explain_prediction_default[0].to_dataframe()

In [ ]:
result_explain_prediction_default[0].show_in_notebook()

<a id='ref'></a>
## References
* More examples and details: http://automl.oraclecorp.com/
* Oracle AutoML http://www.vldb.org/pvldb/vol13/p3166-yakovlev.pdf
* scikit-learn https://scikit-learn.org/stable/
* Interpretable Machine Learning https://christophm.github.io/interpretable-ml-book/
* LIME https://arxiv.org/pdf/1602.04938
* 20newsgroup http://qwone.com/~jason/20Newsgroups/